In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1
!pip install tensorflow

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook

In [4]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
#GPU 사용 시
device = torch.device("cuda:0")

# **딥러닝을 위해 SKT의 KoBERT 이용**

## **데이터 불러오기**

In [6]:
train_path = '/home/lab12/workspace/Insta_User_Profiling/Data_Analysis/Dataset/gender_train_data.csv'
test_path = '/home/lab12/workspace/Insta_User_Profiling/Data_Analysis/Dataset/gender_test_data.csv'

In [7]:
train_data = pd.read_csv(train_path, engine='python', encoding='utf-8', index_col=0)
test_data = pd.read_csv(test_path, engine='python', encoding='utf-8', index_col=0)
len(train_data), len(test_data)

(158754, 39688)

In [8]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(154204, 38515)

In [13]:
dataset_train = [] # 라벨은 0부터 순서대로 입력해야함
for i in tqdm(range(len(train_data))):
    dataset_train.append([train_data['content'][i], int(train_data['label'][i])]) # 해당 리스트 형태를 맞춰야 학습 가능
dataset_train[:5]

100%|██████████| 154204/154204 [00:04<00:00, 32298.34it/s]


[["_𝘽𝙍𝙊𝙒𝘽𝙔𝙅𝙐 ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 고객님들을 위한 소독젤 + 핸드크림 세팅완료 🖤 ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 멀리서 챙겨주는 친구들 덕분에 든든히 소독제품과 보습제 쟁여뒀습니다 😊😊 마음까지 풍족해졌어요❣⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 코로나가 하루 빨리 잠재워지길 바라며.저의 바람, 모두의 바람, 🙏⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 매일 매일 살균 소독하고 있어요 🖤⠀⠀⠀⠀ 안심하고 방문하세요 🖤 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ❛ BROWBYJU ❜ 만의 약속 ⠀ • 일회용품 시술 후 바로 폐기 ෆ • 얼굴형, 피부상태, 골격 등에 맞게 1:1 맞춤시술 ෆ • 프라이빗한 공간에서 시술 ෆ • 최상급 재료 엄선 ෆ • 유지기간 평균 8개월 ~ 1년 6개월 ෆ • 시술 직후 통증이 거의 없는 90% 무통시술 ෆ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ❛ 예약 및 상담 ❜ ⠀ 카카오톡 플러스친구 검색 🔍 '브로바이쥬' 인스타그램 💽 @_brow_by_ju_  ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ",
  1],
 ['이제 집들이 그만할래요.. #집들이 #이사 #빚더미 #친구들 #아그와 #디아블로  #홈스타그램 #칵테일', 0],
 ['정신없이 바쁘더니 몇주가 지났다⠀ 시간이나면 읽어야지 했던 ‘죽음’ 인데 핑계가 많으니 이제서야 손에 쥐었다틈날때마다 조금씩 읽어야지📗 #베르나르베르베르 #죽음 #책 #도서 #독서 #책추천 #책추천해주세요',
  0],
 ['입으면 #하늘하늘 날아갈듯 한 #린넨 소재의 #플라워 #스커트 입니다.  #프로방스 스커트 와 함께 #6월 행복한 한달 되세요  #ootd#oufit#likeforlikes#black#dailylook#daily',
  1],
 ['연주)은채 덕분에 지금 올리는 20년 1월 30일 잠을 10시간씩 자는 고2.. 말도안돼 잠 줄이자🌊 •

In [10]:
dataset_test = []
for i in tqdm(range(len(test_data))):
    dataset_test.append([test_data['content'][i], int(test_data['label'][i])])
dataset_test[:5]

100%|██████████| 38515/38515 [00:01<00:00, 33929.17it/s]


[['고맙습니다:) 열무엄마😘 #아이쿠뭘이런걸 #아까워서쓰겠나🙊 #생일선물 #발렌타인콜라보 #bottegaveneta #보테가 #홍연이생일때어떡하지😔',
  0],
 ['제임스 열무 #007 #고양이 #cat #kitten', 0],
 ['양치기 소치기 심바', 0],
 ['여친이 찍어주는 사진 vs 남친이 찍어주는 사진... 휴....😒 #집들이 #사진 #차이보소 #잘놀다갑니다🙌', 0],
 ['자연광에서 사진찍고싶은데ㅋㅋㅋ요즘 낮에 화장하고 밖에 나갈일이 거의 없응..ㅋㅋㅋ 비타민D 챙겨먹어야지ㅋㅋㅋ', 0]]

## **데이터 토큰화**

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

In [43]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [44]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [45]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [46]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [47]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## **모델링 과정**

In [48]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2, # 해당 부분 파라미터 조정으로 다중 분류 가능
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [49]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [50]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [51]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [52]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [53]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total) # Warm-Up Step 추가

In [54]:
def calc_accuracy(X,Y): # 정확도 계산 함수
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## **이진분류 모델 학습**

In [55]:
PATH = "/home/lab12/workspace/Insta_User_Profiling/Data_Analysis/model_checkpoint/{}_epochs/".format(num_epochs) # 모델 저장 경로 설정

In [56]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    # 모델 학습 : model.train()을 지정해줘야 Fine Tunning이 이뤄짐
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # 모델 평가 : model.eval()을 지정해야 Fine Tunning을 멈추고 평가를 시작함
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # 에폭마다 체크포인트 저장
    torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
    torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
    torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict()
                }, PATH + 'all.tar')

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.6919600367546082 train acc 0.625
epoch 1 batch id 201 loss 0.6227627992630005 train acc 0.6291977611940298
epoch 1 batch id 401 loss 0.5411272048950195 train acc 0.644443578553616
epoch 1 batch id 601 loss 0.5914573073387146 train acc 0.6567179700499168
epoch 1 batch id 801 loss 0.619468629360199 train acc 0.6673299001248439
epoch 1 batch id 1001 loss 0.45616334676742554 train acc 0.6769792707292708
epoch 1 batch id 1201 loss 0.4500175416469574 train acc 0.6843645920066611
epoch 1 batch id 1401 loss 0.6816243529319763 train acc 0.6912138650963597
epoch 1 batch id 1601 loss 0.5014159679412842 train acc 0.6971912086196127
epoch 1 batch id 1801 loss 0.6051709651947021 train acc 0.7034546779566907
epoch 1 batch id 2001 loss 0.29171374440193176 train acc 0.707950712143928
epoch 1 batch id 2201 loss 0.39697372913360596 train acc 0.7126093253066788
epoch 1 batch id 2401 loss 0.41347670555114746 train acc 0.7169148271553519

epoch 1 train acc 0.7170550533491405
/home/


epoch 1 test acc 0.770438407921308


## **모델링 결과 시각화**

In [111]:
# 예측을 위해 Community 데이터 불러오기
predict_test = pd.read_csv('/home/lab12/workspace/Insta_User_Profiling/Data_Estimation/Dataset/community.csv')

predict_test.head()

,content
0,"내 주변엔 좋은사람들만 있어서 행복해막내동생이랑🤪내 코 언제 나아,,,초딩아님‼️1..."
1,마니 컸다..세탁실 앞에서 자는 그는..하루종일 잔다#사기 #보이스피싱 조심하세요...
2,#다시 #짧은 #글귀 #글 #시 #데일리#사진 #빛바랜 #추억 #칠하기#팔로우좋아요...
3,성남시 청년지원센터해피봄스데이🥳🥳.청년이봄 1주년 축하드립니다><코로나19로 인해 ...
4,동대문~땡!ㆍ도매언니/밤시장/ 밴드로 초대합니다.아래를 클릭하시면 밴드 가입 가능하...


In [58]:
predict_dataset = []
for i in tqdm(range(len(predict_test))):
    predict_dataset.append([predict_test['content'][i]])
predict_dataset[0]

100%|██████████| 40/40 [00:00<00:00, 26027.33it/s]


['내 주변엔 좋은사람들만 있어서 행복해막내동생이랑\U0001f92a내 코 언제 나아,,,초딩아님‼️17번째 내 생일💛까오니❤❤랜만에👅9월은 좀 행복했으면 좋겠어,𝓪 𝓰𝓸𝓸𝓭 𝓭𝓪𝔂 𝓽𝓸 𝓻𝓾𝓷 𝓪𝔀𝓪𝔂딱 2년전으로 돌아가고싶다( •́ ̯•̀ )자존감 낮은 요즘.엄마❤담에두❣증맬루 다사다난 했지만 즐거웠똥 내사랑❣️100년만에 카페 💝또 가고파 ღ']

In [59]:
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length=64, pad=True, pair=False)
sentences = [transform(i) for i in predict_dataset]

In [60]:
predict_dataloader = torch.utils.data.DataLoader(sentences, batch_size=64, num_workers=5)

In [61]:
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(predict_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [106]:
# 예측 결과
max_vals, max_indices = torch.max(out, 1)
predict = []
for i in max_indices:
    predict.append(i.item())

predict[:5]

[1, 1, 1, 0, 1]

In [112]:
predict_test['predict_label'] = predict
predict_test.head()

,content,predict_label
0,"내 주변엔 좋은사람들만 있어서 행복해막내동생이랑🤪내 코 언제 나아,,,초딩아님‼️1...",1
1,마니 컸다..세탁실 앞에서 자는 그는..하루종일 잔다#사기 #보이스피싱 조심하세요...,1
2,#다시 #짧은 #글귀 #글 #시 #데일리#사진 #빛바랜 #추억 #칠하기#팔로우좋아요...,1
3,성남시 청년지원센터해피봄스데이🥳🥳.청년이봄 1주년 축하드립니다><코로나19로 인해 ...,0
4,동대문~땡!ㆍ도매언니/밤시장/ 밴드로 초대합니다.아래를 클릭하시면 밴드 가입 가능하...,1


In [131]:
# 예측 결과 확인을 위한 33번 유저 / 실제 남성
predict_test.loc[33]

content          렉에 사람도 없고.. 반신욕기.. 아이좋아..#운동 #헬스 #헬린이앙 왕숙천..이제...
predict_label                                                    0
Name: 33, dtype: object

In [119]:
# 남성과 여성으로 예측한 비율
predict.count(0) / len(predict), predict.count(1) / len(predict)

(0.375, 0.625)

In [113]:
predict_test.to_csv('predict_result.csv')